In [1]:
#!pip install opencv-python
#!apt-get update && apt-get install ffmpeg libsm6 libxext6  -y
#!pip install mediapipe
#!pip install opencv-python transformers accelerate diffusers


In [2]:
import time
t = str(int(time.time()))+"_"
t

'1740409652_'

In [4]:
from diffusers import AutoencoderKL, StableDiffusionXLControlNetPipeline, ControlNetModel, UniPCMultistepScheduler
import torch
import os

from controlnet_aux import OpenposeDetector
from diffusers.utils import load_image
from diffusers import ControlNetModel, StableDiffusionXLControlNetPipeline, AutoencoderKL
from diffusers import DDIMScheduler, EulerAncestralDiscreteScheduler
from controlnet_aux import MidasDetector

from PIL import Image
import torch
import random
import numpy as np
import cv2

In [6]:
controlnet_conditioning_scale = 1.0  
prompt = "A detailed, high-quality photograph, high-quality  realistically, well lit, of a human palm on a pure white background. White background.The palm is facing the viewer, fingers extended, with the pattern of creases and lines on the hand clearly visible. There are no other objects or elements in the image."
negative_prompt = "not a palm, drawn, cartoons, animation, drawing, Blurriness, (many folds), low resolution, other body parts, extraneous objects, background elements, jewelry, tattoos, text, watermarks, multiple hands, gloves, nail polish, distracting details."

controlnet = ControlNetModel.from_pretrained(
    "xinsir/controlnet-depth-sdxl-1.0",
    torch_dtype=torch.float16, cache_dir='./modele cache/'
)

# when test with other base model, you need to change the vae also.
vae = AutoencoderKL.from_pretrained("madebyollin/sdxl-vae-fp16-fix", torch_dtype=torch.float16, cache_dir='./modele cache/')

eulera_scheduler = EulerAncestralDiscreteScheduler.from_pretrained("stabilityai/stable-diffusion-xl-base-1.0", subfolder="scheduler",cache_dir='./modele cache/')


pipe = StableDiffusionXLControlNetPipeline.from_pretrained(
    "John6666/realism-by-stable-yogi-sdxlv2-sdxl",
    controlnet=controlnet,
    vae=vae,
    torch_dtype=torch.float16,
    scheduler=eulera_scheduler,
    cache_dir='./modele cache/'
)# .to("cuda")


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

In [ ]:
%%time
import torch
import os
import random
from PIL import Image
import numpy as np
import mediapipe as mp


# Инициализация MediaPipe Hands
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

hands = mp_hands.Hands(
    static_image_mode=True,        # Режим обработки статичного изображения
    max_num_hands=1,               # Максимальное количество рук для обнаружения
    min_detection_confidence=0.95   # Установите порог для обнаружения
)



for i in range(200):
    # Получаем случайный файл маски
    mask_files = [f for f in os.listdir('./masks/') if f.endswith((".jpg", ".png"))]
    random_mask_file = random.choice(mask_files)
    controlnet_img = Image.open('./masks/' + random_mask_file).convert("L")  # Преобразуем в черно-белый формат
    width, height = controlnet_img.size

    # Генерация изображения с pipe
    images = pipe(
        prompt,
        negative_prompt=negative_prompt,
        image=controlnet_img,
        controlnet_conditioning_scale=controlnet_conditioning_scale,
        width=width,
        height=height,
        num_inference_steps=30,
        num_images_per_prompt=10
    )

    # Конвертируем маску в NumPy массив для применения
    controlnet_mask = np.array(controlnet_img)

    # Пройдем по всем сгенерированным изображениям
    for im in range(len(images.images)):
        #images.images[im].save("./img/" + t + "image_" +random_mask_file[:-4]+"_"+ str(im) + '_' + str(i) + ".jpg")
        ###############################
        # Получаем сгенерированное изображение как NumPy массив
        img_array = np.array(images.images[im])
        # Создаем белый фон, такого же размера как изображение
        white_background = np.full_like(img_array, 255)
        # Применяем маску: где маска черная (0), делаем белым
        masked_image = np.where(controlnet_mask[..., None] <100, white_background, img_array)
        # Конвертируем обратно в PIL изображение
        masked_pil_image = Image.fromarray(masked_image)
        # Сохраняем изображение
        #masked_pil_image.save("./img/" + t + "image_" +random_mask_file[:-4]+"_"+ str(im) + '_' + str(i) + ".jpg")
        #############################

        # Преобразование PIL Image в NumPy массив с цветовым пространством RGB
        image_np = np.array(masked_pil_image.convert('RGB'))
        
        # Обработка изображения
        results = hands.process(image_np)
        
        # Проверка и получение оценки уверенности
        if results.multi_hand_landmarks:
            for idx, hand_landmarks in enumerate(results.multi_hand_landmarks):
                # Получение информации о классификации руки
                hand_classification = results.multi_handedness[idx]
                hand_label = hand_classification.classification[0].label  # 'Left' или 'Right'
                hand_score = hand_classification.classification[0].score  # Оценка уверенности (от 0 до 1)
                probability = hand_score * 100  # Конвертируем в проценты
        
                # Вывод информации
                print(f"вероятность {probability:.2f}%")

                masked_pil_image.save("./img/" + t + "image_" +random_mask_file[:-4]+"_"+ str(im) + '_' + str(i)+f"_{probability:.2f}" + ".jpg")

        else:
            #masked_pil_image.save("./img_b/" + t + "image_" +random_mask_file[:-4]+"_"+ str(im) + '_' + str(i) + ".jpg")
            print("Рука не обнаружена на изображении.")
    
    # Очищаем кэш после каждой итерации
    torch.cuda.empty_cache()

I0000 00:00:1740410054.446164    8842 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1740410054.451797    9523 gl_context.cc:369] GL version: 3.2 (OpenGL ES 3.2 Mesa 24.2.8-1ubuntu1~24.04.1), renderer: AMD Radeon Graphics (radeonsi, renoir, LLVM 19.1.1, DRM 3.57, 6.8.0-53-generic)
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1740410054.481990    9502 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1740410054.518898    9513 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


  0%|          | 0/30 [00:00<?, ?it/s]